In [ ]:
import tensorflow_datasets as tfds
(train, test), info = tfds.load('imdb_reviews', split = (tfds.Split.TRAIN, tfds.Split.TEST), as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.M4YLBT_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.M4YLBT_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
for review, label in tfds.as_numpy(train.take(5)):
  print(review.decode()[0:50], "\t",label)

This was an absolutely terrible movie. Don't be lu 	 0
I have been known to fall asleep during films, but 	 0
Mann photographs the Alberta Rocky Mountains in a  	 0
This is the kind of film for a snowy Sunday aftern 	 1
As others have mentioned, all the women that go nu 	 1


In [ ]:
!pip install -q transformers

In [ ]:
for review, label in tfds.as_numpy(train.take(4)):
  print(review.decode(), "\t",label)
  print("-"*50)

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it. 	 0
--------------------------------------------------
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep beca

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
max_length = 512
batch_size = 6
def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                               add_special_tokens = True, # add [CLS] and [SEP]
                               max_length = max_length, # max length of the text that you can go to BERT
                               pad_to_max_length = True, # Add padd tokens
                               return_attention_mask = True)

In [ ]:
import tensorflow as tf

def map_example_to_dict(input_ids, attention_mask, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_mask
  }, label

def encode_examples(ds, limit=-1):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if(limit > 0):
      ds = ds.take(limit)
    for review, label in tfds.as_numpy(ds):
      bert_input = convert_example_to_feature(review.decode())
      input_ids_list.append(bert_input['input_ids'])
      token_type_ids_list.append(bert_input['token_type_ids'])
      attention_mask_list.append(bert_input['attention_mask'])
      label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [ ]:
train_encoded = encode_examples(train).shuffle(5000).batch(batch_size)
test_encoded  = encode_examples(test).batch(batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_encoded.save('/content/drive/MyDrive/train_encoded.tfrecord')
test_encoded.save('/content/drive/MyDrive/train_encoded.tfrecord')

In [ ]:
loaded_train = tf.data.Dataset.load('/content/drive/MyDrive/train_encoded.tfrecord')
loaded_test = tf.data.Dataset.load('/content/drive/MyDrive/train_encoded.tfrecord')

In [ ]:
from transformers import TFBertForSequenceClassification

learning_rate = 2e-5
epochs = 1 # Higher epoch might led to overfitting
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer = optimizer, loss = loss, metrics = [metric])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_history = model.fit(loaded_train, epochs = epochs, validation_data = loaded_test)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4167/4167 [==============================] - 4234s 996ms/step - loss: 0.2395 - accuracy: 0.9044 - val_loss: 0.0988 - val_accuracy: 0.9676


In [ ]:
def make_prediction(test_sentence, model):

    encoded_sent = tokenizer.encode(test_sentence, truncation = True, padding = True, return_tensors = "tf")
    tf_output = model.predict(encoded_sent)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis = 1)
    labels = ['Negative', 'Positive']
    label = tf.argmax(tf_prediction, axis = 1)
    label = label.numpy()
    return labels[label[0]]

In [ ]:
# model.save('/content/drive/MyDrive/my_model.h5', save_format = "tf")
# If I had to save only model weights:
model.save_weights('/content/drive/MyDrive/my_model_weights.h5')

In [ ]:
# loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.h5')
# Loading only the weights if only weights were saved
model.load_weights('/content/drive/MyDrive/my_model_weights.h5')

In [ ]:
model.compile(optimizer = optimizer, loss = loss, metrics = [metric])

In [ ]:
inp = "It was the best movie"
make_prediction(inp, model)

1/1 [==============================] - 0s 65ms/step


'Positive'